# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 23:34:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.49       Driver Version: 457.49       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   48C    P0    11W /  N/A |    568MiB /  4096MiB |     24%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2021년 Celltrion Data**

In [5]:
lexicon_2021 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2021_celltrion.csv')

lexicon_2021.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,셀트리온,68270,매일경제,2021010106,[돈터뷰] 슈퍼개미에게 묻는다! 사야하는 주식 안되는 주식은?,http://news.mk.co.kr/newsRead.php?no=433&year=...,\n\n\n '보컬'로 알려진 슈퍼개미 김형준 대표는 매일경제 '돈터뷰'에 출연해...,2021-01-04,6,351500,354500,343000,347500,2006483,-0.032033,-1,0,보컬 슈퍼 개미 김형준 대표 매일경제 돈터뷰 출연 자신 투자 철학 방법 공개 대표 ...


In [6]:
lexicon_2021[lexicon_2021['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2021.dropna(axis=0, inplace=True)
lexicon_2021 = lexicon_2021.reset_index(drop=True)
lexicon_2021.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2021 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2021['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2021['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2021['Tokenization'][x].split())):
                    if lexicon_2021['Tokenization'][x].split()[w] == list(set(lexicon_2021['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2021['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2021['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2021 = pos_lexicon_2021.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2021['date'][x], positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z])))

***2021-01-04 Cosine Similarity between <강세> & <급락> : 0.507910966873169
***2021-01-04 Cosine Similarity between <해결> & <방법> : 0.5185781121253967
***2021-01-04 Cosine Similarity between <강세> & <급락> : 0.507910966873169
***2021-01-04 Cosine Similarity between <해결> & <방법> : 0.5185781121253967
***2021-01-04 Cosine Similarity between <추진> & <검토> : 0.5113311409950256
***2021-01-04 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2021-01-04 Cosine Similarity between <중요한> & <가장> : 0.5371253490447998
***2021-01-04 Cosine Similarity between <증대> & <감소> : 0.6517642736434937
***2021-01-04 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2021-01-04 Cosine Similarity between <확보> & <마련> : 0.5137236714363098
***2021-01-04 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2021-01-04 Cosine Similarity between <중요한> & <가장> : 0.5371253490447998
***2021-01-04 Cosine Similarity between <증대> & <감소> : 0.6517642736434937
***2021-01-04 Cosine Similarity between <증대> & <증가> :

In [11]:
pos_lexicon_2021

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2021-01-04,1,강세,급락,0.507911,2
1,2021-01-04,1,해결,방법,0.518578,5
2,2021-01-04,2,강세,급락,0.507911,2
3,2021-01-04,2,해결,방법,0.518578,5
4,2021-01-04,3,추진,검토,0.511331,1
...,...,...,...,...,...,...
7042,2021-09-15,1299,증대,확대,0.519324,1
7043,2021-09-15,1299,증대,확보,0.516144,2
7044,2021-09-15,1299,추진,검토,0.511331,1
7045,2021-09-15,1299,추진,사업,0.521374,1


In [12]:
pos_lexicon_2021.to_csv('../../../../Code/Data/Test/Stock-Year/pos_celltrion_2021.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2021['news_word'])))

a

['언택트',
 '자공시',
 '덴티움덴티움',
 '증대',
 '최선',
 '텍오스코텍',
 '발전',
 '증진',
 '펀인터렉티브',
 '웃돈',
 '특성',
 '검토',
 '계획',
 '의미',
 '강점',
 '트윈데믹',
 '유데나필',
 '임희진',
 '언택트주',
 '방안',
 '급락',
 '사트칼리예프',
 '침체',
 '견조할',
 '닛케이지수',
 '검체체취',
 '마그비',
 '단점',
 '충족',
 '이뮤노메딕스',
 '노인환',
 '박유악',
 '기록',
 '요구',
 '웃돌',
 '중인',
 '횡보세',
 '밑돈',
 '방법',
 '장단점',
 '아달리무맙',
 '들보',
 '랙티브',
 '듀센바예프',
 '허쥬마뿐',
 '특징',
 '촉진',
 '폭락',
 '케이에스',
 '확립',
 '안트로젠안트로젠',
 '루브리컨츠',
 '디지털리서치',
 '텍티앤에이텍',
 '엠에스',
 '향상',
 '감소',
 '듀비에정',
 '주요한',
 '니케이지수',
 '인플렉트',
 '스탁액티브',
 '레그단비맙',
 '전개',
 '진행',
 '강세',
 '훈풍',
 '심핵폴딩스',
 '칸진티',
 '우호',
 '문제',
 '룩시엔스',
 '이점',
 '해법',
 '레그단미밥',
 '확대',
 '추진',
 '자산성',
 '띠었음',
 '가장',
 '리레이팅',
 '팽팽하',
 '법정상속',
 '맹폭격',
 '게임스탑',
 '가치나',
 '밑돌',
 '펜데믹',
 '추동훈',
 '극복',
 '더브릿지',
 '리메드',
 '가야쿠',
 '조아연',
 '최진명',
 '안트로퀴노놀',
 '약세',
 '심텍심텍',
 '박승영',
 '지성규',
 '증가',
 '맥쿼리인프라',
 '상쇄하',
 '경의',
 '본격',
 '나정환',
 '유동물량',
 '소재효',
 '실패',
 '시너지',
 '장점',
 '딘퓨쳐스',
 '급증',
 '활기',
 '셀루메드셀루메드',
 '확충',
 '중요',
 '강력',
 '크래프톤크래프톤',
 '윤재춘',
 '활황',
 '적극',
 '확

In [14]:
b = list(set(list(pos_lexicon_2021[pos_lexicon_2021['cosine_similarity']>=0.7]['news_word'])))

b

['셀루메드셀루메드', '약세', '개선', '최선', '밑돈', '실패', '단점', '추진', '충족', '장점', '경의']

In [15]:
c = list(set(list(pos_lexicon_2021[pos_lexicon_2021['cosine_similarity']>=0.65]['news_word'])))

c

['약세',
 '최선',
 '지성규',
 '웃돌',
 '증가',
 '경의',
 '트윈데믹',
 '밑돈',
 '실패',
 '추진',
 '장점',
 '셀루메드셀루메드',
 '안트로젠안트로젠',
 '적극',
 '개선',
 '감소',
 '단점',
 '충족',
 '이뮤노메딕스']